# Anomaly Detection Project

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn import metrics
from env import user, host, password
import os
import wrangle as w

In [3]:
df = w.get_cohort_logs_data()

In [4]:
df.head()

,date,time,path,user_id,cohort_id,ip,id,name,slack,start_date,end_date,created_at,updated_at,deleted_at,program_id
0,2018-01-26,09:55:03,/,1,8.0,97.105.19.61,8.0,Hampton,#hampton,2015-09-22,2016-02-06,2016-06-14 19:52:26,2016-06-14 19:52:26,NaN,1.0
1,2018-01-26,09:56:02,java-ii,1,8.0,97.105.19.61,8.0,Hampton,#hampton,2015-09-22,2016-02-06,2016-06-14 19:52:26,2016-06-14 19:52:26,NaN,1.0
2,2018-01-26,09:56:05,java-ii/object-oriented-programming,1,8.0,97.105.19.61,8.0,Hampton,#hampton,2015-09-22,2016-02-06,2016-06-14 19:52:26,2016-06-14 19:52:26,NaN,1.0
3,2018-01-26,09:56:06,slides/object_oriented_programming,1,8.0,97.105.19.61,8.0,Hampton,#hampton,2015-09-22,2016-02-06,2016-06-14 19:52:26,2016-06-14 19:52:26,NaN,1.0
4,2018-01-26,09:56:24,javascript-i/conditionals,2,22.0,97.105.19.61,22.0,Teddy,#teddy,2018-01-08,2018-05-17,2018-01-08 13:59:10,2018-01-08 13:59:10,NaN,2.0


In [22]:
df.info()
# drop 'deleted_at'
# rows that didn't have a 'cohort_id' have nulls

<class 'pandas.core.frame.DataFrame'>
Int64Index: 900223 entries, 0 to 900222
Data columns (total 15 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   date        900223 non-null  object 
 1   time        900223 non-null  object 
 2   path        900222 non-null  object 
 3   user_id     900223 non-null  int64  
 4   cohort_id   847330 non-null  float64
 5   ip          900223 non-null  object 
 6   id          847330 non-null  float64
 7   name        847330 non-null  object 
 8   slack       847330 non-null  object 
 9   start_date  847330 non-null  object 
 10  end_date    847330 non-null  object 
 11  created_at  847330 non-null  object 
 12  updated_at  847330 non-null  object 
 13  deleted_at  0 non-null       float64
 14  program_id  847330 non-null  float64
dtypes: float64(4), int64(1), object(10)
memory usage: 109.9+ MB


In [44]:
no_cid = df[df.cohort_id.isna()]
#selects rows without a cohort_id

In [46]:
no_cid.user_id.unique()
# some sort of codeup association

array([ 48,  54,  58,  59,  61,  62,  63,  73,  74,  78,  79,  86,  88,
        89, 100, 103, 111, 137, 166, 176, 213, 247, 317, 349, 350, 351,
       352, 353, 354, 355, 356, 357, 358, 359, 360, 361, 362, 363, 364,
       365, 366, 367, 368, 369, 372, 375, 403, 406, 429, 544,  64, 713,
       714, 715, 716, 717, 718, 663, 719, 720, 721, 722, 723, 724, 725,
       726, 727, 728, 729, 346, 731, 736, 644, 744, 782, 810, 814, 815])

In [53]:
for i in no_cid.user_id.unique():
    if df[df.user_id == 48].cohort_id.isna().mean() < 1:
        print(i)

In [7]:
df.isna().sum()

date               0
time               0
path               1
user_id            0
cohort_id      52893
ip                 0
id             52893
name           52893
slack          52893
start_date     52893
end_date       52893
created_at     52893
updated_at     52893
deleted_at    900223
program_id     52893
dtype: int64

In [21]:
df.path.unique().shape
# 2314 unique paths

(2314,)

In [30]:
df.path.value_counts().head(20)
#look only at top 10 paths

/                                                                            50313
search/search_index.json                                                     19519
javascript-i                                                                 18983
toc                                                                          18297
java-iii                                                                     13733
html-css                                                                     13635
java-ii                                                                      12685
spring                                                                       12524
jquery                                                                       11525
mysql                                                                        11033
java-i                                                                       10865
javascript-ii                                                                10730
appe

In [25]:
df['path_1'] = df.path.str.split('/').str[0]

In [27]:
df['path_2'] = df.path.str.split('/').str[1]

In [31]:
df['path_3'] = df.path.str.split('/').str[2]

In [33]:
df['path_4'] = df.path.str.split('/').str[3]

In [40]:
df['path_5'] = df.path.str.split('/').str[4]

In [41]:
df.head()

,date,time,path,user_id,cohort_id,ip,id,name,slack,start_date,end_date,created_at,updated_at,deleted_at,program_id,path_1,path_2,path_3,path_4,path_5
0,2018-01-26,09:55:03,/,1,8.0,97.105.19.61,8.0,Hampton,#hampton,2015-09-22,2016-02-06,2016-06-14 19:52:26,2016-06-14 19:52:26,NaN,1.0,,,NaN,NaN,NaN
1,2018-01-26,09:56:02,java-ii,1,8.0,97.105.19.61,8.0,Hampton,#hampton,2015-09-22,2016-02-06,2016-06-14 19:52:26,2016-06-14 19:52:26,NaN,1.0,java-ii,NaN,NaN,NaN,NaN
2,2018-01-26,09:56:05,java-ii/object-oriented-programming,1,8.0,97.105.19.61,8.0,Hampton,#hampton,2015-09-22,2016-02-06,2016-06-14 19:52:26,2016-06-14 19:52:26,NaN,1.0,java-ii,object-oriented-programming,NaN,NaN,NaN
3,2018-01-26,09:56:06,slides/object_oriented_programming,1,8.0,97.105.19.61,8.0,Hampton,#hampton,2015-09-22,2016-02-06,2016-06-14 19:52:26,2016-06-14 19:52:26,NaN,1.0,slides,object_oriented_programming,NaN,NaN,NaN
4,2018-01-26,09:56:24,javascript-i/conditionals,2,22.0,97.105.19.61,22.0,Teddy,#teddy,2018-01-08,2018-05-17,2018-01-08 13:59:10,2018-01-08 13:59:10,NaN,2.0,javascript-i,conditionals,NaN,NaN,NaN


In [42]:
df[df.path_5.isna() == False]

,date,time,path,user_id,cohort_id,ip,id,name,slack,start_date,end_date,created_at,updated_at,deleted_at,program_id,path_1,path_2,path_3,path_4,path_5
2071,2018-01-30,11:37:48,content/laravel/intro/gitbook/images/favicon.ico,68,13.0,216.1.153.162,13.0,Kings,#kings,2016-05-23,2016-09-15,2016-06-14 19:52:26,2016-06-14 19:52:26,NaN,1.0,content,laravel,intro,gitbook,images
2073,2018-01-30,11:37:49,content/laravel/intro/gitbook/images/favicon.ico,68,13.0,216.1.153.162,13.0,Kings,#kings,2016-05-23,2016-09-15,2016-06-14 19:52:26,2016-06-14 19:52:26,NaN,1.0,content,laravel,intro,gitbook,images
2075,2018-01-30,11:37:50,content/laravel/quickstart/gitbook/images/favi...,68,13.0,216.1.153.162,13.0,Kings,#kings,2016-05-23,2016-09-15,2016-06-14 19:52:26,2016-06-14 19:52:26,NaN,1.0,content,laravel,quickstart,gitbook,images
2078,2018-01-30,11:38:31,content/laravel/intro/gitbook/images/favicon.ico,68,13.0,216.1.153.162,13.0,Kings,#kings,2016-05-23,2016-09-15,2016-06-14 19:52:26,2016-06-14 19:52:26,NaN,1.0,content,laravel,intro,gitbook,images
2080,2018-01-30,11:38:33,content/laravel/intro/gitbook/images/favicon.ico,68,13.0,216.1.153.162,13.0,Kings,#kings,2016-05-23,2016-09-15,2016-06-14 19:52:26,2016-06-14 19:52:26,NaN,1.0,content,laravel,intro,gitbook,images
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
881764,2021-04-08,15:05:29,appendix/open_data/www.who.int/research/en,837,133.0,66.69.94.59,133.0,Easley,#easley,2020-12-07,2021-06-08,2020-12-07 15:20:18,2020-12-07 15:20:18,NaN,3.0,appendix,open_data,www.who.int,research,en
881780,2021-04-08,15:13:10,appendix/open_data/www.who.int/research/en,837,133.0,66.69.94.59,133.0,Easley,#easley,2020-12-07,2021-06-08,2020-12-07 15:20:18,2020-12-07 15:20:18,NaN,3.0,appendix,open_data,www.who.int,research,en
882420,2021-04-09,00:10:17,content/html-css/gitbook/images/favicon.ico,80,14.0,136.50.29.193,14.0,Lassen,#lassen,2016-07-18,2016-11-10,2016-07-18 19:06:27,2016-07-18 19:06:27,NaN,1.0,content,html-css,gitbook,images,favicon.ico
882426,2021-04-09,00:10:29,content/javascript/gitbook/images/favicon.ico,80,14.0,136.50.29.193,14.0,Lassen,#lassen,2016-07-18,2016-11-10,2016-07-18 19:06:27,2016-07-18 19:06:27,NaN,1.0,content,javascript,gitbook,images,favicon.ico
